In [1]:
import urllib2
import ssl

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

links = [ "https://199.119.112.144/onionperf-one-51200-2017-01-19.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-20.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-21.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-22.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-23.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-24.tpf",         
        ]

days = []

for link in links:
    f = urllib2.urlopen(link, context=ctx)
    myfile = f.read()
    days.append(myfile)

In [14]:
from datetime import datetime

class DataPoint: 
    def __init__(self, iterable=[]):
        self.data = {}      
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            key = item[0]
            value = item[1]
            self.data[key]=value

    __update = update   # private copy of original update() method
    
    def setStr(self, s, d):
        self.data[s] = str(d)
        
    def setFilesize(self, d):
        self.data["FILESIZE"] = int(d)
    
    def setTimeStamp(self, s, d):
        self.data[s] = datetime.fromtimestamp(float(d))

    def setWritebytes(self, d):
        self.data["WRITEBYTES"] = int(d)
    
    def setReadbytes(self, d):
        self.data["READBYTES"] = int(d)
        
    def setDidtimeout(self, d):
        if d == '1':
            self.data["DIDTIMEOUT"] = True
        elif d == '0':
            self.data["DIDTIMEOUT"] = False
    
    def setPath(self, d):
        self.data["PATH"] = d.split(',')
        
    def setBuildTimes(self, d):
        bt = d.split(',')
        self.data["BUILDTIMES"] = [float(t) for t in bt]
        
    def setTimeout(self, d):
        self.data["TIMEOUT"] = int(d)
        
    def setQuantile(self, d):
        self.data["QUANTILE"] = float(d)
    
    def setCircId(self, d):
        self.data["CIRC_ID"] = int(d)
        
    def setUsedBy(self, d):
        self.data["USED_BY"] = int(d)
        
    def setData(self, k, d):
        selectors = {"SOURCE": lambda d: self.setStr("SOURCE", d),
                     "FILESIZE": lambda d: self.setFilesize(d),
                     "START": lambda d: self.setTimeStamp("START", d),
                     "SOCKET": lambda d: self.setTimeStamp("SOCKET", d),
                     "CONNECT": lambda d: self.setTimeStamp("CONNECT", d),
                     "NEGOTIATE": lambda d: self.setTimeStamp("NEGOTIATE", d),
                     "REQUEST": lambda d: self.setTimeStamp("REQUEST", d),
                     "RESPONSE": lambda d: self.setTimeStamp("RESPONSE", d),
                     "DATAPERC0": lambda d: self.setTimeStamp("DATAPER0", d), 
                     "DATAPERC10": lambda d: self.setTimeStamp("DATAPER10", d),  
                     "DATAPERC100": lambda d: self.setTimeStamp("DATAPER100", d), 
                     "DATAPERC20": lambda d: self.setTimeStamp("DATAPER20", d), 
                     "DATAPERC30": lambda d: self.setTimeStamp("DATAPER30", d), 
                     "DATAPERC40": lambda d: self.setTimeStamp("DATAPER40", d), 
                     "DATAPERC50": lambda d: self.setTimeStamp("DATAPER50", d), 
                     "DATAPERC60": lambda d: self.setTimeStamp("DATAPER60", d), 
                     "DATAPERC70": lambda d: self.setTimeStamp("DATAPER70", d), 
                     "DATAPERC80": lambda d: self.setTimeStamp("DATAPER80", d), 
                     "DATAPERC90": lambda d: self.setTimeStamp("DATAPER90", d),  
                     "DATAREQUEST": lambda d: self.setTimeStamp("DATAREQUEST", d), 
                     "DATARESPONSE": lambda d: self.setTimeStamp("DATARESPONSE", d), 
                     "DATACOMPLETE": lambda d: self.setTimeStamp("DATACOMPLETE", d), 
                     "WRITEBYTES": lambda d: self.setWritebytes(d),
                     "READBYTES": lambda d: self.setReadbytes(d),
                     "DIDTIMEOUT": lambda d: self.setDidtimeout(d),
                     "LAUNCH": lambda d: self.setTimeStamp("LAUNCH", d), 
                     "USED_AT": lambda d: self.setTimeStamp("USED_AT", d), 
                     "PATH": lambda d: self.setPath(d),
                     "BUILDTIMES": lambda d: self.setBuildTimes(d),
                     "TIMEOUT": lambda d: self.setTimeout(d),
                     "QUANTILE": lambda d: self.setQuantile(d),
                     "CIRC_ID": lambda d: self.setCircId(d),
                     "USED_BY": lambda d: self.setUsedBy(d),
                     "ENDPOINTLOCAL": lambda d: self.setStr("ENDPOINTLOCAL", d), 
                     "ENDPOINTPROXY": lambda d: self.setStr("ENDPOINTPROXY", d), 
                     "ENDPOINTREMOTE": lambda d: self.setStr("ENDPOINTREMOTE", d),
                     "HOSTNAMELOCAL": lambda d: self.setStr("HOSTNAMELOCAL", d),
                     "HOSTNAMEREMOTE": lambda d: self.setStr("HOSTNAMEREMOTE", d)}
            
        return selectors[k](d)
        

In [16]:
day_ps = []
for text in days:
    data = text.split('@type torperf 1.0')
    data_points = []
    for d in data:
        dp = d.split(' ')
        data_points.append(dp)
    data_points.pop(0)

    dps = []
    for dp in data_points:
        p = DataPoint()
        connect = 0
        complete = 0
        timeout = 0
        for d in dp:
            f = d.split('=')
            key = f[0]
            value = f[1]
            if '\r\n' in f[0]:
                key = f[0].split('\r\n')[1]
            p.setData(key, value)   
        
        dps.append(p.data)
    day_ps.append(dps)


In [41]:
import numpy as np

deltas = []

for day in day_ps:
    t = []
    for dp in dps:
        transfer = dp['DATAPER100'] - dp['DATAPER0']
        t.append(float(transfer.microseconds)/float(100000))
    deltas.append(t)

t = np.median(deltas, axis=0)

In [43]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

M = len(links)
# prepare some data
N = len(deltas)
x = np.linspace(0, M, N)
# output to static HTML file
output_file("transfer_times.html")

# create a new plot
s1 = figure(width=2048, plot_height=1024, tools=TOOLS)
s1.line(x, t, line_dash=(1, 2), line_color="green", line_width=2)
# NEW: put the subplots in a gridplot
p = gridplot([[s1]], toolbar_location=None)

# show the results
show(p)

/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
